# [DC学院\_员工离职预测训练赛](http://www.dcjingsai.com/common/cmpt/%E5%91%98%E5%B7%A5%E7%A6%BB%E8%81%8C%E9%A2%84%E6%B5%8B%E8%AE%AD%E7%BB%83%E8%B5%9B_%E7%BB%93%E6%9E%9C%E6%8F%90%E4%BA%A4.html)

## 数据探索

In [2]:
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
import numpy as np

In [2]:
lr = linear_model.LogisticRegression()

In [3]:
resignation = pd.read_csv('E:/MySQL_data/DataCastle/pfm_train.csv',encoding='utf-8')

In [ ]:
resignation.describe().columns

In [ ]:
resignation.columns

In [ ]:
resignation.sample(5)

## 使用已有标签

In [15]:
features = ['Age','DistanceFromHome','Education','EmployeeNumber',
            'EnvironmentSatisfaction','JobInvolvement','JobLevel',
            'JobSatisfaction', 'MonthlyIncome', 'NumCompaniesWorked',
            'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
            'StandardHours', 'StockOptionLevel', 'TotalWorkingYears',
            'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
            'YearsInCurrentRole', 'YearsSinceLastPromotion',
            'YearsWithCurrManager']
x = resignation[features]
y = resignation['Attrition']

这里的features是resignation.describe().columns

In [36]:
from sklearn.tree import DecisionTreeClassifier

In [38]:
dt = DecisionTreeClassifier()
score_dt = cross_val_score(dt,x,y,cv=5,scoring='accuracy').mean()
score_dt

0.74553709787392308

In [58]:
from sklearn import neighbors

best_num = 1
knn_best_num = neighbors.KNeighborsClassifier(best_num,weights='distance')
best_score = cross_val_score(knn_best_num,x,y,cv=5,scoring='accuracy').mean()
for i in range(2,10):
    knn_i = neighbors.KNeighborsClassifier(i,weights='distance')
    score_i = cross_val_score(knn_i,x,y,cv=5,scoring='accuracy').mean()
    if score_i > best_score:
        best_score = score_i
        best_num = i

print('最佳近邻数量：{}，准确率{}'.format(best_num,best_score))

最佳近邻数量：7，准确率0.8218194027299587


In [74]:
knn_best_num = neighbors.KNeighborsClassifier(best_num,weights='distance')

In [59]:
from sklearn import ensemble

best_deep = 1
rf_best_deep = ensemble.RandomForestClassifier(best_deep)
best_score = cross_val_score(rf_best_deep,x,y,cv=5,scoring='accuracy').mean()
for deep in range(2,24):
    rf_deep = ensemble.RandomForestClassifier(deep)
    score_deep = cross_val_score(rf_deep,x,y,cv=5,scoring='accuracy').mean()
    if score_deep > best_score:
        best_score = score_deep
        best_deep = deep

print('最佳深度：{}，准确率{}'.format(best_deep,best_score))

最佳深度：19，准确率0.8482165108595405


In [73]:
rf_best_deep = ensemble.RandomForestClassifier(best_deep)

In [61]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
score_gnb = cross_val_score(gnb,x,y,cv=5,scoring='accuracy').mean()
score_gnb

0.71910704390962243

In [ ]:
from sklearn.linear_model.stochastic_gradient import SGDClassifier
sgdc = SGDClassifier()
score_sgdc = cross_val_score(sgdc,x,y,cv=5,scoring='accuracy').mean()
score_sgdc

In [ ]:
from sklearn.linear_model.perceptron import Perceptron
pct = Perceptron()
score_pct = cross_val_score(pct,x,y,cv=5,scoring='accuracy').mean()
score_pct

In [70]:
from sklearn.svm import SVC
svc = SVC()
score_svc = cross_val_score(svc,x,y,cv=5,scoring='accuracy').mean()
score_svc

0.8381874155927338

In [72]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()
score_lsvc = cross_val_score(lsvc,x,y,cv=5,scoring='accuracy').mean()
score_lsvc

0.83456750609047137

In [76]:
models = [lsvc,svc,pct,sgdc,gnb,rf_best_deep,knn_best_num,dt,lr]
model_df = pd.DataFrame(np.zeros((len(models),2)),columns=['model','score'])

for i in range(len(models)):
    model_df.loc[i,'model'] = str(models[i]).split('(')[0]
    model_df.loc[i,'score'] = cross_val_score(models[i],x,y,cv=5,
                                              scoring='accuracy').mean() 

model_df.sort_values(by='score',ascending=False)

,model,score
8,LogisticRegression,0.852712
5,RandomForestClassifier,0.840006
1,SVC,0.838187
0,LinearSVC,0.836377
2,Perceptron,0.822745
6,KNeighborsClassifier,0.821819
7,DecisionTreeClassifier,0.744669
4,GaussianNB,0.719107
3,SGDClassifier,0.702114


不同模型在处理同一数据时的能力不同，选择最优秀的模型进行预测

In [78]:
model_lr = lr.fit(x,y)

In [3]:
resignation_test = pd.read_csv('E:\\MySQL_data\\DataCastle\\pfm_test.csv',
                               encoding='utf-8')

In [77]:
x_test = resignation_test[features]
pfm_predict = model_lr.predict(x_test)

In [35]:
pd.DataFrame(pfm_predict).to_csv('E:\\MySQL_data\\DataCastle\\pfm_predict.csv',
                                index = False,
                                header = ['result'],
                                encoding = 'utf-8')

第1次提交到DC后得分：0.87428，排名142<br>
![第1次提交](https://github.com/incipient1/resignation_predict/blob/master/img/score_1_dc.PNG)

## 标签化9列

### 标签化BusinessTravel

BusinessTravel|商务差旅频率
--------------|----------
Non-Travel    |不出差
Travel_Rarely |不经常出差
Travel_Frequently|经常出差

In [80]:
set(resignation['BusinessTravel'])

{'Non-Travel', 'Travel_Frequently', 'Travel_Rarely'}

In [81]:
resignation['business_travel'] = resignation['BusinessTravel']

In [82]:
resignation.loc[resignation['business_travel'] == 'Non-Travel','business_travel'] = 0
resignation.loc[resignation['business_travel'] == 'Travel_Rarely','business_travel'] = 1
resignation.loc[resignation['business_travel'] == 'Travel_Frequently','business_travel'] = 2  

In [83]:
set(resignation['business_travel'])

{0, 1, 2}

标签化成功

### 标签化Department

Department|员工所在部门
-----|-----
Sales|销售部
Research & Development|研发部
Human Resources|人力资源部

标签和部门之间没有关系，故自动标签

In [84]:
resignation['department'] = resignation['Department']

In [87]:
set(resignation['department'])

{'Human Resources', 'Research & Development', 'Sales'}

In [88]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(resignation['department'])
resignation['department'] = le.transform(resignation['department'])

### 标签化EducationField

In [91]:
set(resignation['EducationField'])

{'Human Resources',
 'Life Sciences',
 'Marketing',
 'Medical',
 'Other',
 'Technical Degree'}

In [92]:
resignation['education_field'] = resignation['EducationField']

In [93]:
le.fit(resignation['education_field'])
resignation['education_field'] = le.transform(resignation['education_field'])

### 标签化Gender

In [96]:
resignation['gender'] = resignation['Gender']
resignation.loc[resignation['gender'] == 'Female','gender'] = 0
resignation.loc[resignation['gender'] == 'Male','gender'] = 1

### 标签化JobRole

In [107]:
set(resignation['job_role'])

{0, 1, 2, 3, 4, 5, 6, 7, 8}

In [102]:
resignation['job_role'] = resignation['JobRole']
le.fit(resignation['job_role'])
resignation['job_role'] = le.transform(resignation['job_role'])

### 标签化MaritalStatus

In [113]:
set(resignation['marital_status'])

{0, 1, 2}

In [104]:
resignation['marital_status'] = resignation['MaritalStatus']
le.fit(resignation['marital_status'])
resignation['marital_status'] = le.transform(resignation['marital_status'])

### 标签化OverTime

In [109]:
resignation['over_time'] = resignation['OverTime']

In [111]:
le.fit(resignation['over_time'])
resignation['over_time'] = le.transform(resignation['over_time'])

## 新标签训练

In [138]:
new_labels = ['business_travel','department','education_field','gender','job_role',
             'marital_status','over_time']
features.extend(new_labels)

In [136]:
x_wrap = resignation[list(set(features))]

In [137]:
models = [lsvc,svc,pct,sgdc,gnb,rf_best_deep,knn_best_num,dt,lr]
model_df = pd.DataFrame(np.zeros((len(models),2)),columns=['model','score'])

for i in range(len(models)):
    model_df.loc[i,'model'] = str(models[i]).split('(')[0]
    model_df.loc[i,'score'] = cross_val_score(models[i],x_wrap,y,cv=5,
                                              scoring='accuracy').mean() 

model_df.sort_values(by='score',ascending=False)

,model,score
8,LogisticRegression,0.874486
5,RandomForestClassifier,0.854576
1,SVC,0.838187
2,Perceptron,0.838187
3,SGDClassifier,0.838187
6,KNeighborsClassifier,0.818183
7,DecisionTreeClassifier,0.783658
4,GaussianNB,0.754550
0,LinearSVC,0.721499


最优模型lr提高了0.02分：从0.85提高到0.87，效果不明显

## 新标签预测

### 标签化，同上

In [5]:
columns_old = ['Department','EducationField','JobRole','MaritalStatus',
               'OverTime']

columns_new = ['department','education_field','job_role','marital_status',
               'over_time']

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in range(len(columns_old)):
    resignation_test[columns_new[i]] = resignation_test[columns_old[i]]
    le.fit(resignation_test[columns_new[i]])
    resignation_test[columns_new[i]] = le.transform(
        resignation_test[columns_old[i]])


,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,JobInvolvement,...,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,department,education_field,job_role,marital_status,over_time
129,52,Travel_Rarely,Sales,5,3,Life Sciences,1319,2,Male,3,...,2,8,7,7,7,2,1,7,2,1
135,59,Travel_Rarely,Sales,3,3,Life Sciences,1254,3,Female,2,...,3,6,1,0,5,2,1,7,2,0
156,31,Travel_Rarely,Research & Development,7,4,Life Sciences,76,4,Male,3,...,2,7,7,1,7,1,1,2,0,0
345,24,Travel_Rarely,Sales,10,4,Marketing,507,4,Female,3,...,4,5,2,0,3,2,2,7,0,1
51,31,Non-Travel,Research & Development,3,2,Medical,1948,3,Male,3,...,4,4,2,2,3,1,3,6,0,1


In [141]:
resignation_test['business_travel'] = resignation_test['BusinessTravel']
resignation_test.loc[resignation_test['business_travel'] == 
                     'Non-Travel','business_travel'] = 0
resignation_test.loc[resignation_test['business_travel'] == 
                     'Travel_Rarely','business_travel'] = 1
resignation_test.loc[resignation_test['business_travel'] == 
                     'Travel_Frequently','business_travel'] = 2  

resignation_test['gender'] = resignation_test['Gender']
resignation_test.loc[resignation_test['gender'] == 'Female','gender'] = 0
resignation_test.loc[resignation_test['gender'] == 'Male','gender'] = 1

### 预测

In [142]:
x_test_wrap = resignation_test[list(set(features))]

In [143]:
model_lr_wrap = lr.fit(x_wrap,y)

In [144]:
y_test_wrap = model_lr_wrap.predict(x_test_wrap)
pd.DataFrame(y_test_wrap).to_csv('E:\\MySQL_data\\DataCastle\\pfm_predict_wrap.csv',
                                index = False,
                                header = ['result'],
                                encoding = 'utf-8')

第2次提交后得分：0.88857，提高了0.01分；排名：85，提高了57<br>
![第2次得分](https://github.com/incipient1/resignation_predict/blob/master/img/score_2_dc.PNG)